In [15]:
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel, BertTokenizer, BertModel, BertConfig
import torch
import torch.nn as nn
import os

In [16]:
import numpy as np

def save_to(base, x, y):
    np_input = x.detach().numpy()
    with open(os.path.join(base, 'dummy_in.dat'), 'wb') as f:
        np_input.tofile(f)

    np_output = y.detach().numpy()
    with open(os.path.join(base, 'dummy_out.dat'), 'wb') as f:
        np_output.tofile(f)

In [17]:
torch.manual_seed(2333)
torch.set_num_threads(1)

In [24]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)
model.config.return_dict = False
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

In [28]:
print(model)
model.summary()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

AttributeError: 'GPT2LMHeadModel' object has no attribute 'summary'

In [30]:
output = model.generate(**encoded_input, max_length=128)
print(tokenizer.decode(output[0]))

Replace me by any text you'd like.

I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware


In [9]:
class GPT2_Component(nn.Module):
    def __init__(self, gpt2: GPT2Model):
        super().__init__()
        self.wte = gpt2.wte
        self.wpe = gpt2.wpe
        self.h0 = gpt2.h[0]
    
    def forward(self, src: Tensor) -> Tensor:
        

Sequential(
  (0): Embedding(50257, 768)
  (1): Embedding(1024, 768)
  (2): GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D()
      (c_proj): Conv1D()
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D()
      (c_proj): Conv1D()
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)


In [11]:
print(encoded_input)

{'input_ids': tensor([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [12]:
x = torch.randint(0, 50257, (64, 20))
x.dtype

torch.int64

In [13]:
y = gpt2_components(x)
y.size()

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [14]:
input_names = ["text"]
output_names = ["prediction"]
dummy_input = x = torch.randint(0, 50257, (64, 20))
torch.onnx.export(model,
                  dummy_input,
                  "./gpt2.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [4]:
bert = BertModel.from_pretrained('bert-base-uncased')
bert.eval()
print(bert)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [10]:
x = torch.randint(0, 30522, (64, 512))

In [11]:
%%time
y = bert(x)
y

CPU times: user 2min 52s, sys: 29.1 s, total: 3min 21s
Wall time: 3min 21s


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2851,  0.4922,  0.1198,  ..., -0.6141,  0.0716, -0.0431],
         [-0.3825,  0.6201,  0.2818,  ..., -0.7040, -0.0581, -0.1138],
         [-0.1337,  0.6262,  0.1319,  ..., -0.6431, -0.1216,  0.0900],
         ...,
         [ 0.2188,  0.2675,  0.1719,  ..., -0.5769, -0.0906, -0.3819],
         [ 0.3521,  0.2902,  0.1557,  ..., -0.5920,  0.0389, -0.4749],
         [ 0.2103,  0.3638,  0.1671,  ..., -0.5304,  0.0427, -0.4705]],

        [[ 0.3828,  0.3511,  0.2953,  ..., -0.4501,  0.0874, -0.0315],
         [-0.2216,  0.7184,  0.4956,  ..., -0.4140,  0.2895,  0.4578],
         [-0.4479,  1.1966,  0.2802,  ..., -0.5680,  0.1829,  0.3152],
         ...,
         [ 0.1119,  0.5961,  0.6280,  ..., -0.5744, -0.1669, -0.2146],
         [ 0.2082,  0.5817,  0.5419,  ..., -0.6793, -0.1145, -0.1381],
         [ 0.3426,  0.4707,  0.5258,  ..., -0.5269, -0.1410, -0.0705]],

        [[-0.0362,  0.1214,  0.3569,  ..., -0.0865,  

In [5]:
input_names = ["text"]
output_names = ["prediction"]
dummy_input = torch.randint(0, 30522, (64, 20))

torch.onnx.export(bert,
                  dummy_input,
                  "./bert.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [10]:
save_to('.', x, y.last_hidden_state)